In [15]:
from email.headerregistry import Address
from unicodedata import category
import pandas as pd
melb_data = pd.read_csv('data/melb_data_ps.csv', sep = ',')
melb_df = melb_data.copy()

melb_df = melb_df.drop(['index', 'Coordinates'], axis=1)

total_rooms = melb_df['Bathroom'] + melb_df['Rooms'] + melb_df['Bedroom']
melb_df['MeanRoomsSquare'] = melb_df['BuildingArea'] / total_rooms

diff_area = melb_df['BuildingArea'] - melb_df['Landsize']
sum_area = melb_df['BuildingArea'] + melb_df['Landsize']
melb_df['AreaRatio'] = diff_area/sum_area

melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
#display(melb_df['Date'])
melb_df['WeekdaySale'] = melb_df['Date'].dt.day_of_week
weekend_count = melb_df[(melb_df['WeekdaySale'] == 5) | (melb_df['WeekdaySale'] == 6)].shape[0]
#display(weekend_count)
#display(melb_df['WeekdaySale'])

# Преобразование столбцов с помощью функций
def get_street_type(address):
    exclude_list = ['N', 'S', 'W', 'E']
    address_list = address.split(' ')
    street_type = address_list[-1]
    if street_type in exclude_list:
        street_type = address_list[-2]
    return street_type

street_types = melb_df['Address'].apply(get_street_type)
#display(street_types)
#print(street_types.nunique()) 
#street_types.value_counts()   
popular_stypes =street_types.value_counts().nlargest(10).index
#print(popular_stypes)
melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')
#display(melb_df['StreetType'])
melb_df['StreetType'].nunique()
melb_df = melb_df.drop('Address', axis=1)

def get_weekend(weekday):
    if weekday == 5 or weekday == 6:
        return 1
    else:
        return 0
    
melb_df['Weekend'] = melb_df['WeekdaySale'].apply(get_weekend)
#round(melb_df[melb_df['Weekend'] == 1]['Price'].mean())


best_sellers = melb_df['SellerG'].value_counts().nlargest(49).index
melb_df['SellerG'] = melb_df['SellerG'].apply(lambda x: x if x in best_sellers else 'other')
min_price_Nelson = melb_df[melb_df['SellerG'] == 'Nelson']['Price'].min()
display(min_price_Nelson)
min_price_other = melb_df[melb_df['SellerG'] == 'other']['Price'].min()
display(min_price_other)
dif_price = round(min_price_Nelson/min_price_other, 2)
#print(dif_price)
#display(melb_df['SellerG'].value_counts())


# КАТЕГОРИИ

#определяем число уникальных категорий в каждом столбце
# создаём пустой список
unique_list = []
# пробегаемся по именам столбцов в таблице
for col in melb_df.columns:
    # создаём кортеж (имя столбца, число уникальных значений)
    item = (col, melb_df[col].nunique(),melb_df[col].dtypes) 
    # добавляем кортеж в список
    unique_list.append(item) 
# создаём вспомогательную таблицу и сортируем её
unique_counts = pd.DataFrame(
    unique_list,
    columns=['Column_Name', 'Num_Unique', 'Type']
).sort_values(by='Num_Unique',  ignore_index=True)
# выводим её на экран
#display(unique_counts)

#преобразование некоторых столбцов к типу данных category
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца
#display(melb_df.info())


sub_categ = melb_df['Suburb'].value_counts().nlargest(119).index
print(sub_categ)
melb_df['Suburb'] = melb_df['Suburb'].apply(lambda x: x if x in sub_categ else 'other')

melb_df['Suburb'] = melb_df['Suburb'].astype('category')
display(melb_df['Suburb'])
display(melb_df.info())

170000.0

131000.0

Index(['Reservoir', 'Richmond', 'Bentleigh East', 'Preston', 'Brunswick',
       'Essendon', 'South Yarra', 'Glen Iris', 'Hawthorn', 'Coburg',
       ...
       'Albion', 'Hoppers Crossing', 'Sunbury', 'Greensborough', 'Bundoora',
       'Hughesdale', 'Chadstone', 'Caulfield North', 'Mont Albert',
       'Alphington'],
      dtype='object', length=119)


0          Abbotsford
1          Abbotsford
2          Abbotsford
3          Abbotsford
4          Abbotsford
             ...     
13575           other
13576    Williamstown
13577    Williamstown
13578    Williamstown
13579      Yarraville
Name: Suburb, Length: 13580, dtype: category
Categories (120, object): ['Abbotsford', 'Aberfeldie', 'Airport West', 'Albert Park', ..., 'Williamstown', 'Windsor', 'Yarraville', 'other']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 25 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  category      
 1   Rooms            13580 non-null  int64         
 2   Type             13580 non-null  category      
 3   Price            13580 non-null  float64       
 4   Method           13580 non-null  category      
 5   SellerG          13580 non-null  category      
 6   Date             13580 non-null  datetime64[ns]
 7   Distance         13580 non-null  float64       
 8   Postcode         13580 non-null  int64         
 9   Bedroom          13580 non-null  int64         
 10  Bathroom         13580 non-null  int64         
 11  Car              13580 non-null  int64         
 12  Landsize         13580 non-null  float64       
 13  BuildingArea     13580 non-null  float64       
 14  YearBuilt        13580 non-null  categ

None

In [36]:
countries_df = pd.DataFrame({
    'country': ['Англия', 'Канада', 'США', 'Россия', 'Украина', 'Беларусь', 'Казахстан'],
    'population': [56.29, 38.05, 322.28, 146.24, 45.5, 9.5, 17.04],
    'square': [133396, 9984670, 9826630, 17125191, 603628, 207600, 2724902]
})
countries_df['density'] = (countries_df['population'] / countries_df['square']) * 1000000
print(round((countries_df['density'].mean()), 2))
#display(countries_df['density'])

84.93


In [30]:

import pandas as pd
df = pd.read_csv('data/ufo.csv', sep = ',')
#ufo_data.info()
#ufo_data.shape
df['Time'] = pd.to_datetime(df['Time'], dayfirst=False)
#years_view = ufo_data['Time'].dt.year
#print(years_view.mode())
df['Date'] = df['Time'].dt.date
print(df[df['State']=='NV']['Date'].diff().dt.days.mean())
#ufo_data.head()

68.92932862190813


In [90]:
import pandas as pd

def get_experience(arg):
    """
    Напишите функцию get_experience(arg), аргументом которой является строка столбца с опытом работы. 
    Функция должна возвращать опыт работы в месяцах. Не забудьте привести результат к целому числу.
    """
    experience = int(0)
    list_month = ['месяц', 'месяцев', 'месяца']
    list_year = ['год', 'года', 'лет']
    list_arg = arg.split(" ")
    print(list_arg)
    if list_arg[-1] in list_month:
        experience += int(list_arg[-2])
    else:
        if list_arg[-1] in list_year:
            experience = experience +int(list_arg[-2])*12
            return experience
    if list_arg[-3] in list_year:
        experience = experience + int(list_arg[-4])*12
        return experience
    else:
        return experience
    
      

experience_col = pd.Series([
        'Опыт работы 8 лет 3 месяца',
        'Опыт работы 3 года 5 месяцев',
        'Опыт работы 1 год 9 месяцев',
        'Опыт работы 3 месяца',
        'Опыт работы 6 лет'
        ])
experience_month = experience_col.apply(get_experience)
print(experience_month)

['Опыт', 'работы', '8', 'лет', '3', 'месяца']
['Опыт', 'работы', '3', 'года', '5', 'месяцев']
['Опыт', 'работы', '1', 'год', '9', 'месяцев']
['Опыт', 'работы', '3', 'месяца']
['Опыт', 'работы', '6', 'лет']
0    99
1    41
2    21
3     3
4    72
dtype: int64


In [117]:
import pandas as pd
bike_df = pd.read_csv("data/citibike-tripdata.csv")
#bike_df.info()
#display(round(bike_df['start station id'].mode()[0])) #Найдите идентификатор самой популярной стартовой стоянки. Запишите идентификатор в виде целого числа.
#bike_df['bikeid'].mode() #Велосипед с каким идентификатором является самым популярным?
#bike_df['usertype']. value_counts(normalize= True) #Какой тип клиентов (столбец usertype) является преобладающим — Subscriber или Customer? 
#bike_df['gender'].value_counts()
#display(bike_df['start station id'].nunique())
#display(bike_df['end station id'].nunique())
#bike_df['birth year'].max()
#display(bike_df['end station name'].value_counts())
bike_df.drop(['start station id', 'end station id'], axis=1, inplace=True)
bike_df['age'] = 2018 - bike_df['birth year']
bike_copy = bike_df #копия базы
bike_df.drop(['birth year'], axis=1, inplace=True)
#bike_df[bike_df['age'] > 60].shape[0] #Сколько поездок совершено клиентами старше 60 лет?
bike_df['starttime'] = pd.to_datetime(bike_df['starttime'])
bike_df['stoptime'] = pd.to_datetime(bike_df['stoptime'])
bike_df['trip duration'] = bike_df['stoptime'] - bike_df['starttime']
#bike_df['trip duration'][3]
#Создайте «признак-мигалку» weekend, который равен 1, если поездка начиналась в выходной день 
#(суббота или воскресенье), и 0 — в противном случае. Выясните, сколько поездок начиналось в выходные.
day_week = bike_df['starttime'].dt.day_of_week
#display(day_week)
def get_wekeend(day_week):
    if days == 5 or days == 6:
        return 1
    else:
        return 0
        
bike_df['weekend'] = day_week.apply(get_weekend)
#display(bike_df['weekend'].value_counts())


trip_time = bike_df['starttime'].dt.hour
def get_time(time):
    if 0 <= time <= 6:
        return 'night'
    if 6 < time <= 12:
        return 'morning'
    if 12 < time <= 18:
        return 'day'
    if 18 < time <=23:
        return 'evening'
    else:
        return 'else'
bike_df['time_of_day'] = trip_time.apply(get_time)     
display(bike_df['time_of_day'].value_counts())
day_way= bike_df[bike_df['time_of_day'] == 'day'].shape[0]
night_way = bike_df[bike_df['time_of_day'] == 'night'].shape[0]
dif_=round(day_way/night_way)
display(day_way)
display(night_way)
display(dif_)


day        143012
morning     95530
evening     46373
night       15085
Name: time_of_day, dtype: int64

143012

15085

9